In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
# import PIL
import tensorflow as tf
import pickle5 as pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob
import random
import datetime

In [358]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [359]:
# import pathlib
# data_dir = pathlib.Path('E:/Users/sentinel_industry/downloaded_aois/')


In [2]:
# x_size = 230
# y_size = 230
# ch1_mean, ch1_std = 970.4162, 1051.8454
# ch2_mean, ch2_std = 1176.249, 1031.262
# ch3_mean, ch3_std = 1273.2376, 1117.068
# ch4_mean, ch4_std = 2266.9050, 1241.5509
# means = [ch1_mean,ch2_mean,ch3_mean,ch4_mean]
# vari = [ch1_std, ch2_std, ch3_std, ch4_std]

# def load_features(name):
#     decoded = name.decode("UTF-8")
#     if os.path.exists(decoded):
#         with open(decoded, 'rb') as f:
#             file = pickle.load(f)
#             label = tf.strings.split(tf.strings.split(name, '/')[-1], '\\')[-2]
#             if label == 'coal':
#                 label = [1,0,0]
#             elif label == 'steel':
#                 label = [0,1,0]
#             else: label = [0,0,1]
#             if (file["B02"].shape[1]<230 or file["B02"].shape[2]<230): 
#                 print("oh oh, downloaded patch too smol:", decoded)
#             B02, B03, B04, B08 = file['B02'][0][0:x_size,0:y_size], file['B03'][0][0:x_size,0:y_size], file['B04'][0][0:x_size,0:y_size],file['B08'][0][0:x_size,0:y_size]
#             B02 = (B02-ch1_mean)/ch1_std
#             B03 = (B03-ch2_mean)/ch2_std
#             B04 = (B04-ch3_mean)/ch3_std
#             B08 = (B08-ch4_mean)/ch4_std
#             features = np.array([ B02, B03, B04, B08]).transpose(1,2,0)
#             # features = np.expand_dims(features, axis=0)
#             return features, label
#             # I have commented the line below but this should return
#             # the features and the label in a one hot vector
#             # return file['features'], file['targets']
#     else:
#         print("Something went wrong!")
#         exit(-1)
        
# def data_loader(filename):
#     features, labels = tf.numpy_function(load_features, [filename], [tf.double, tf.int32])
#     # features.set_shape((None, 242,242,4))
#     # labels.set_shape(( 1))
#     return features, labels

In [3]:
valid = tf.data.experimental.load('savedata_valid')
train = tf.data.experimental.load('savedata_train')

In [4]:
train = train.shuffle(100,  reshuffle_each_iteration=True)
# valid = valid.shuffle(100,)


In [5]:
# dataset = dataset.map((
#   lambda x, y: (resize_and_rescale(x), y)))

In [31]:
x_size = 230
y_size = 230
in_seed = 42
resize_rescale_augment = tf.keras.Sequential([
    layers.RandomRotation(0.2, seed=in_seed, input_shape=(x_size,y_size,4)),  # here, maybe try larger values
    layers.CenterCrop(140,140, ),
    layers.RandomCrop(120,120, seed=in_seed, ),  # to be changed with centercrop for data augmentation/ use centre first, then random?
    
    # layers.Rescaling(1./10000),#,input_shape=(240,240,4)), # I think the data is scaled to 10000
    layers.RandomFlip("horizontal_and_vertical", seed=in_seed), 
    layers.Conv2D(3, 5, padding='same', activation='softmax'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                                # maybe try changing the source so it accepts 4 channels?
])

In [26]:
import keras_tuner
from keras_tuner.applications import HyperResNet
from keras_tuner.tuners import Hyperband

hypermodel = HyperResNet(include_top=False,
        # weights=None, # if I don't use the pre trained weights from image net, does it matter that i don't use the preprocessing step which reorders RGB to BGR and zero-centers wrt imagenet?
        input_shape=(120, 120, 3),
        # pooling=max ,
        classes=3,)


In [40]:
hypermodel.build()

TypeError: _build_wrapper() missing 1 required positional argument: 'hp'

In [41]:
tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    # max_trials=20,
    directory='hyperres',
    project_name='sentinel_industry')

tuner.search(
    train, epochs=1, validation_data=valid
)

INFO:tensorflow:Reloading Oracle from existing project hyperres\sentinel_industry\oracle.json


ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [20]:
    model = tf.keras.Sequential([
        resize_rescale_augment,
        layers.Conv2D(3, 5, padding='same', activation='softmax'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                                # maybe try changing the source so it accepts 4 channels?
        hypermodel,
        layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
        layers.Dense(3)
    ])

    log_dir = "E:/Users/sentinel_industry/logs/train/new/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '120,140')

    def scheduler(epoch, lr):
        if epoch < 30:
            return lr
        else:
            return lr * tf.math.exp(-0.05)

    lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return model

In [21]:
build_model(keras_tuner.HyperParameters())

TypeError: The added layer must be an instance of class Layer. Received: layer=<keras_tuner.applications.resnet.HyperResNet object at 0x000002132E590488> of type <class 'keras_tuner.applications.resnet.HyperResNet'>.

In [520]:
model.summary()

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_40 (Sequential)  (None, 120, 120, 4)       0         
                                                                 
 conv2d_32 (Conv2D)          (None, 120, 120, 3)       303       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_25 (Flatten)        (None, 32768)             0         
                                                                 
 dense_27 (Dense)            (None, 3)                 98307     
                                                                 
Total params: 23,686,322
Trainable params: 23,633,202
Non-trainable params: 53,120
_________________________________________________________________


In [480]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [436]:
# !env LD_LIBRARY_PATH="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# os.environ['LD_LIBRARY_PATH']="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# os.environ['LD_INCLUDE_PATH']="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# # !set LD_INCLUDE_PATH=:/usr/local/cuda/include:/usr/local/cuda/extras/CUPTI/include

In [437]:
!env LD_LIBRARY_PATH

env: â€˜LD_LIBRARY_PATHâ€™: No such file or directory


In [393]:
%tensorboard --logdir='E:/Users/sentinel_industry/logs/train/new'

Reusing TensorBoard on port 6006 (pid 22096), started 2 days, 1:46:10 ago. (Use '!kill 22096' to kill it.)

In [521]:
# now with normalisation
epochs=100
batch_size = 40
history = model.fit(
    train.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_data=valid.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_steps=10,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_callback]
)

Epoch 1/100
159/159 [==============================] - 105s 590ms/step - loss: 2.8663 - categorical_accuracy: 0.5514 - val_loss: 0.9951 - val_categorical_accuracy: 0.5250 - lr: 0.0010
Epoch 2/100
159/159 [==============================] - 63s 396ms/step - loss: 1.8282 - categorical_accuracy: 0.6307 - val_loss: 1.8915 - val_categorical_accuracy: 0.5175 - lr: 0.0010
Epoch 3/100
159/159 [==============================] - 65s 403ms/step - loss: 1.2748 - categorical_accuracy: 0.6574 - val_loss: 1.4815 - val_categorical_accuracy: 0.5400 - lr: 0.0010
Epoch 4/100
159/159 [==============================] - 64s 400ms/step - loss: 1.3946 - categorical_accuracy: 0.6428 - val_loss: 1.0628 - val_categorical_accuracy: 0.7125 - lr: 0.0010
Epoch 5/100
159/159 [==============================] - 64s 399ms/step - loss: 0.8354 - categorical_accuracy: 0.7167 - val_loss: 0.8240 - val_categorical_accuracy: 0.6300 - lr: 0.0010
Epoch 6/100
159/159 [==============================] - 65s 403ms/step - loss: 1.0920

KeyboardInterrupt: 

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(18, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
iterator=tf.compat.v1.data.make_one_shot_iterator(dataset)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    next_element = iterator.get_next()
    im = resize_rescale_augment(next_element[0])
    plt.imshow(np.flip(im[0,:,:,0:3], axis=2))
    plt.title(convert_onehot(next_element))

In [ ]:
def convert_onehot(next_element):
    if (next_element[1].numpy() == np.array([0, 0, 1])).all(): return 'no industry'
    elif (next_element[1].numpy() == np.array([0, 1, 0])).all(): return 'steel'
    elif (next_element[1].numpy() == np.array([1, 0, 0])).all(): return 'coal'
    else: return 'not valid'

In [ ]:
next_element[1].numpy().all()

In [ ]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("module.name", "/path/to/file.py")
foo = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = foo
spec.loader.exec_module(foo)

In [ ]:
! PATH